# Notebook for MSLP Data Preprocessing

By: Ty Janoski

Updated 1/14/2026

## Setup

### Imports

In [17]:
# Import Statements
import cartopy.crs as ccrs
import cmweather  # noqa: F401
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scienceplots  # noqa: F401
import xarray as xr

plt.style.use(["science", "nature", "grid"])
plt.rcParams["text.usetex"] = True

### Data Read-In

#### CSV

In [3]:
# Read in CSV file with flash flood events
df = pd.read_csv("../data/storm_data_search_results.csv")


#### ERA5 Output

In [9]:
# # Read in hourly, sliced total precipitation (tp) data
# ds = xr.open_mfdataset(
#     "/mnt/drive2/ERA5/NC_files/hourly_sliced/era5_mslp_*.nc",
#     combine="by_coords",
#     decode_times=True,
#     chunks={"time": 8760},
# )

# # Save out
# with ProgressBar():
#     ds.load()

# ds.msl.rename("mslp").to_netcdf(
#     "/mnt/drive2/ERA5/NC_files/combined/era5_mslp_hourly_warm_season_US.nc"
# )

mslp = xr.load_dataarray(
    "/mnt/drive2/ERA5/NC_files/combined/era5_mslp_hourly_warm_season_US.nc",
    decode_timedelta=True,
)


## Data Processing

### Flash Flood Times

In [10]:
# Remove rows where EVENT_ID is not a digit
df = df[df["EVENT_ID"].astype(str).str.isdigit()]

# Turn BEGIN_TIME and END_TIME into strings with leading zeros if necessary
df["BEGIN_TIME"] = df["BEGIN_TIME"].fillna(0).astype(int).astype(str).str.zfill(4)
df["END_TIME"] = df["END_TIME"].fillna(0).astype(int).astype(str).str.zfill(4)

# Combine TIME and DATE into a single datetime string
begin_str = df["BEGIN_DATE"] + " " + df["BEGIN_TIME"] # type: ignore
end_str = df["END_DATE"] + " " + df["END_TIME"] # type: ignore

# Convert the datetime strings to pandas datetime objects
df["BEGIN_DATETIME"] = pd.to_datetime(
    begin_str, format="%m/%d/%Y %H%M", errors="coerce"
)
df["END_DATETIME"] = pd.to_datetime(end_str, format="%m/%d/%Y %H%M", errors="coerce")

# Take only the first row for each EPISODE_ID
df_unique = df.drop_duplicates(subset=["EPISODE_ID"], keep="first").copy()

# Create a list of datetimes rounded to the nearest preceding hour
event_hours = df_unique["BEGIN_DATETIME"].dt.floor("h").tolist() # type: ignore

In [15]:
# Group by day of year to calculate mean and std dev
mean_doy = mslp.groupby("valid_time.dayofyear").mean(dim="valid_time")
std_doy = mslp.groupby("valid_time.dayofyear").std(dim="valid_time")

std_doy_smooth = (
    std_doy
    .sortby("dayofyear")
    .rolling(dayofyear=14, center=True, min_periods=7)
    .mean()
)

# Calculate standardized anomalies
mslp_anoms = mslp.groupby("valid_time.dayofyear") - mean_doy
mslp_norm  = mslp_anoms.groupby("valid_time.dayofyear") / std_doy_smooth

# Weight by square root of cosine of latitude
weights = np.sqrt(np.cos(np.deg2rad(mslp_norm.latitude)))
mslp_norm_weighted = mslp_norm * weights

In [21]:
(mslp/100).isel(valid_time=0).max()

<xarray.DataArray 'mslp' ()> Size: 4B
array(1028.21, dtype=float32)
Coordinates:
    valid_time  datetime64[ns] 8B 1996-05-01
Attributes: (12/32)
    standard_name:                            air_pressure_at_mean_sea_level
    long_name:                                Mean sea level pressure
    units:                                    Pa
    GRIB_paramId:                             151
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      260281
    ...                                       ...
    GRIB_missingValue:                        3.4028234663852886e+38
    GRIB_name:                                Mean sea level pressure
    GRIB_shortName:                           msl
    GRIB_totalNumber:                         0
    GRIB_units:                               Pa
    GRIB_surface:                             0.0

In [27]:
# Create an example plot of the standardized anomalies
fig, axs = plt.subplots(
    1, 3, figsize=(6.5, 3.5), subplot_kw={"projection": ccrs.PlateCarree()}, dpi=600
)

# Define plot configurations
configs = [
    {
        "data": mslp.isel(valid_time=0) / 100,
        "lon": mslp.longitude,
        "lat": mslp.latitude,
        "title": "MSLP",
        "cmap": "viridis",
        "levels": np.arange(988, 1032, 4),
    },
    {
        "data": mslp_norm.isel(valid_time=0),
        "lon": mslp_norm.longitude,
        "lat": mslp_norm.latitude,
        "title": "MSLP Standardized Anomalies",
        "cmap": "balance",
        "levels": np.arange(-3, 3.1, 0.5),
    },
    {
        "data": mslp_norm_weighted.isel(valid_time=0),
        "lon": mslp_norm_weighted.longitude,
        "lat": mslp_norm_weighted.latitude,
        "title": "MSLP Standardized Anomalies Weighted",
        "cmap": "balance",
        "levels": np.arange(-3, 3.1, 0.5),
    },
]

# Create plots and colorbars
for i, config in enumerate(configs):
    c = axs[i].contourf(
        config["lon"],
        config["lat"],
        config["data"],
        cmap=config["cmap"],
        levels=config["levels"],
        extend="both"
    )
    axs[i].set_title(config["title"], fontsize=8)
    axs[i].coastlines(resolution="50m", linewidth=0.5)

    cb = fig.colorbar(c, ax=axs[i], orientation="horizontal", pad=0.03)
    if i == 0:
        cb.set_label("mb", fontsize=6)
    cb.set_ticks(c.levels)
    cb.ax.tick_params(labelsize=6, rotation=45)

plt.tight_layout()
plt.savefig(
    "../figs/mslp-SOM/mslp_standardized_anomalies_example.png",
    dpi=600,
    bbox_inches="tight",
)
plt.close()


### Intersecting Times

In [28]:
# Extract data for event hours
# The times are actual NYC local, so we need to convert them to UTC
times_local = pd.to_datetime(event_hours).tz_localize("US/Eastern")
times_utc = times_local.tz_convert("UTC").tz_convert(None)

intersect = pd.Index(times_utc).intersection(mslp_norm_weighted.indexes["valid_time"])
mslp_norm_weighted_ffe = mslp_norm_weighted.sel(valid_time=intersect)
mslp_norm_ffe = mslp_norm.sel(valid_time=intersect)
mslp_ffe = mslp.sel(valid_time=intersect)


## Save out data

In [29]:
# Save out the processed data
mslp_norm_weighted_ffe.to_netcdf(
    "/mnt/drive2/SOM_intermediate_files/era5_mslp_norm_weighted_ffe.nc"
)
mslp_norm_ffe.to_netcdf("/mnt/drive2/SOM_intermediate_files/era5_mslp_norm_ffe.nc")
mslp_ffe.to_netcdf("/mnt/drive2/SOM_intermediate_files/era5_mslp_ffe.nc")

In [30]:
# Save out daily averaged raw Z500 and unweighted standardized anomalies
mslp_daily = (
    mslp.resample(valid_time="1D", label="left", closed="left")
    .mean()
    .dropna(dim="valid_time", how="all")
)
mslp_norm_daily = (
    mslp_norm.resample(valid_time="1D", label="left", closed="left")
    .mean()
    .dropna(dim="valid_time", how="all")
)
mslp_norm_weighted_daily = (
    mslp_norm_weighted.resample(valid_time="1D", label="left", closed="left")
    .mean()
    .dropna(dim="valid_time", how="all")
)

mslp_daily.to_netcdf("/mnt/drive2/SOM_intermediate_files/era5_mslp_daily.nc")
mslp_norm_daily.to_netcdf("/mnt/drive2/SOM_intermediate_files/era5_mslp_norm_daily.nc")
mslp_norm_weighted_daily.to_netcdf(
    "/mnt/drive2/SOM_intermediate_files/era5_mslp_norm_weighted_daily.nc"
)
